# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up Graphlab Create

In [1]:
import graphlab

In [2]:
import numpy as np

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [3]:
sales = graphlab.SFrame('kc_house_data.gl/')

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\suveyday\AppData\Local\Temp\graphlab_server_1484664127.log.0


This non-commercial license of GraphLab Create for academic use is assigned to suveydayeniterzi@gmail.com and will expire on January 03, 2018.


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [4]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [5]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [6]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features, validation_set=None, l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 1.572282     | 6869667.895833     | 385998.458623 |

| 6         | 7        | 0.000002  | 1.615312     | 6847177.773672     | 380824.455891 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [7]:
model_all.get("coefficients").print_rows(num_rows=18)

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None |  274873.05595 |  None  |
|     bedrooms     |  None |      0.0      |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 8468.53108691 |  None  |
|   sqft_living    |  None | 24.4207209824 |  None  |
| sqft_living_sqrt |  None | 350.060553386 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None | 842.068034898 |  None  |
|    sqft_above    |  None | 20.0247224171 |  None  |
|  sqft_basement   |  None |

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

In [8]:
for i in range(len(model_all.get("coefficients"))):
    if model_all.coefficients['value'][i] != 0:
        print model_all.coefficients['name'][i]

(intercept)
bathrooms
sqft_living
sqft_living_sqrt
grade
sqft_above


# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [9]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [10]:
l1_penalty_list = np.logspace(1, 7, num=13)

In [11]:
min_RSS = 2**63 - 1
RSS_list = []

for l1_penalty in l1_penalty_list:
    
    model = graphlab.linear_regression.create(training, target='price', features=all_features, validation_set=None, l2_penalty=0., l1_penalty=l1_penalty, verbose = False)
    
    validation_results = model.predict(validation)
    validation_RSS = ((validation['price'] - validation_results) **2).sum()
    print "The RSS of the validation data using the l1_penalty = " + str(l1_penalty) + " is " + str(validation_RSS)
    
    RSS_list.append(validation_RSS)
    
    # Updating the minimum RSS and the best model
    if validation_RSS < min_RSS:
        min_RSS = validation_RSS
        best_l1_penalty = l1_penalty

print "The lowest RSS of the validation data (" + str(min_RSS) + ") is obtained using the l1_penalty = " + str(best_l1_penalty) 

The RSS of the validation data using the l1_penalty = 10.0 is 6.25766285142e+14
The RSS of the validation data using the l1_penalty = 31.6227766017 is 6.25766285362e+14
The RSS of the validation data using the l1_penalty = 100.0 is 6.25766286058e+14
The RSS of the validation data using the l1_penalty = 316.227766017 is 6.25766288257e+14
The RSS of the validation data using the l1_penalty = 1000.0 is 6.25766295212e+14
The RSS of the validation data using the l1_penalty = 3162.27766017 is 6.25766317206e+14
The RSS of the validation data using the l1_penalty = 10000.0 is 6.25766386761e+14
The RSS of the validation data using the l1_penalty = 31622.7766017 is 6.25766606749e+14
The RSS of the validation data using the l1_penalty = 100000.0 is 6.25767302792e+14
The RSS of the validation data using the l1_penalty = 316227.766017 is 6.25769507644e+14
The RSS of the validation data using the l1_penalty = 1000000.0 is 6.25776517727e+14
The RSS of the validation data using the l1_penalty = 316227

*** QUIZ QUESTION. *** What was the best value for the `l1_penalty`?

In [12]:
print "The best value for the l1_penalty is " + str(best_l1_penalty) 

The best value for the l1_penalty is 10.0


***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [13]:
model = graphlab.linear_regression.create(training, target='price', features=all_features, validation_set=None, l2_penalty=0., l1_penalty=best_l1_penalty, verbose = False)
        
print "This model has " + str(model['coefficients']['value'].nnz()) + " nonzero weights"        

This model has 18 nonzero weights


# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [14]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [15]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [16]:
nonzero_list = []

for l1_penalty in l1_penalty_values:
    
    model = graphlab.linear_regression.create(training, target='price', features=all_features, validation_set=None, l2_penalty=0., l1_penalty=l1_penalty, verbose = False)
    
    nonzero_list.append(model['coefficients']['value'].nnz())

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

***QUIZ QUESTION.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 

In [17]:
l1_penalty_min = l1_penalty_values[len(filter(lambda x: x > 7, nonzero_list)) - 1]
l1_penalty_max = l1_penalty_values[len(l1_penalty_values) - len(filter(lambda x: x < 7, nonzero_list))]

print "l1_penalty_min = " + str(l1_penalty_min)
print "l1_penalty_max = " + str(l1_penalty_max)

l1_penalty_min = 2976351441.63
l1_penalty_max = 3792690190.73


## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [18]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [19]:
min_RSS = 2**63 - 1
RSS_list = []

for l1_penalty in l1_penalty_values:
    
    model = graphlab.linear_regression.create(training, target='price', features=all_features, validation_set=None, l2_penalty=0., l1_penalty=l1_penalty, verbose = False)
        
    validation_results = model.predict(validation)
    validation_RSS = ((validation['price'] - validation_results) **2).sum()
    print "The RSS of the validation data using the l1_penalty = " + str(l1_penalty) +  " is " + str(validation_RSS)
    
    RSS_list.append(validation_RSS)
    
    # Updating the minimum RSS and the best model
    if validation_RSS < min_RSS:
        min_RSS = validation_RSS
        best_l1_penalty = l1_penalty

print "The lowest RSS of the validation data (" + str(min_RSS) + ") is obtained using the l1_penalty = " + str(best_l1_penalty)

The RSS of the validation data using the l1_penalty = 2976351441.63 is 9.66925692362e+14
The RSS of the validation data using the l1_penalty = 3019316638.95 is 9.74019450085e+14
The RSS of the validation data using the l1_penalty = 3062281836.27 is 9.81188367942e+14
The RSS of the validation data using the l1_penalty = 3105247033.59 is 9.89328342459e+14
The RSS of the validation data using the l1_penalty = 3148212230.92 is 9.98783211266e+14
The RSS of the validation data using the l1_penalty = 3191177428.24 is 1.00847716702e+15
The RSS of the validation data using the l1_penalty = 3234142625.56 is 1.01829878055e+15
The RSS of the validation data using the l1_penalty = 3277107822.88 is 1.02824799221e+15
The RSS of the validation data using the l1_penalty = 3320073020.2 is 1.03461690923e+15
The RSS of the validation data using the l1_penalty = 3363038217.52 is 1.03855473594e+15
The RSS of the validation data using the l1_penalty = 3406003414.84 is 1.04323723787e+15
The RSS of the validat

In [20]:
final_model = graphlab.linear_regression.create(training, target='price', features=all_features, validation_set=None, l2_penalty=0., l1_penalty=best_l1_penalty, verbose = False)
final_model.get("coefficients").print_rows(num_rows=18)        

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None | 196100.937806 |  None  |
|     bedrooms     |  None | 2181.57432107 |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 17962.6966612 |  None  |
|   sqft_living    |  None | 34.1424656512 |  None  |
| sqft_living_sqrt |  None | 789.319789078 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |  3665.9308176 |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None | 11333.8410308 |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None | 3578.90040044 |  None  |
|    sqft_above    |  None | 32.7432013718 |  None  |
|  sqft_basement   |  None |

As you can see this model has more than 7 nonzero weights. Therefore, we need to add a constraint to the above code to make sure the optimum model has 7 nonzero weights.

In [21]:
min_RSS = 2**63 - 1
RSS_list = []

for l1_penalty in l1_penalty_values:
    
    model = graphlab.linear_regression.create(training, target='price', features=all_features, validation_set=None, l2_penalty=0., l1_penalty=l1_penalty, verbose = False)
        
    validation_results = model.predict(validation)
    validation_RSS = ((validation['price'] - validation_results) **2).sum()
    print "The RSS of the validation data using the l1_penalty = " + str(l1_penalty) + " is " + str(validation_RSS)
    
    RSS_list.append(validation_RSS)
    
    # Updating the minimum RSS and the best model
    if validation_RSS < min_RSS and model['coefficients']['value'].nnz() == max_nonzeros:
        min_RSS = validation_RSS
        best_l1_penalty = l1_penalty

print "The lowest RSS of the validation data (" + str(min_RSS) + ") is obtained using the l1_penalty = " + str(best_l1_penalty)

The RSS of the validation data using the l1_penalty = 2976351441.63 is 9.66925692362e+14
The RSS of the validation data using the l1_penalty = 3019316638.95 is 9.74019450085e+14
The RSS of the validation data using the l1_penalty = 3062281836.27 is 9.81188367942e+14
The RSS of the validation data using the l1_penalty = 3105247033.59 is 9.89328342459e+14
The RSS of the validation data using the l1_penalty = 3148212230.92 is 9.98783211266e+14
The RSS of the validation data using the l1_penalty = 3191177428.24 is 1.00847716702e+15
The RSS of the validation data using the l1_penalty = 3234142625.56 is 1.01829878055e+15
The RSS of the validation data using the l1_penalty = 3277107822.88 is 1.02824799221e+15
The RSS of the validation data using the l1_penalty = 3320073020.2 is 1.03461690923e+15
The RSS of the validation data using the l1_penalty = 3363038217.52 is 1.03855473594e+15
The RSS of the validation data using the l1_penalty = 3406003414.84 is 1.04323723787e+15
The RSS of the validat

In [22]:
final_model = graphlab.linear_regression.create(training, target='price', features=all_features, validation_set=None, l2_penalty=0., l1_penalty=best_l1_penalty, verbose = False)
final_model.get("coefficients").print_rows(num_rows=18)        

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None | 222253.192544 |  None  |
|     bedrooms     |  None | 661.722717782 |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 15873.9572593 |  None  |
|   sqft_living    |  None | 32.4102214513 |  None  |
| sqft_living_sqrt |  None | 690.114773313 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None | 2899.42026975 |  None  |
|    sqft_above    |  None | 30.0115753022 |  None  |
|  sqft_basement   |  None |

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [23]:
print "The best value for the l1_penalty is " + str(best_l1_penalty) + " which leads to " + str(final_model['coefficients']['value'].nnz()) + " nonzero weights"

The best value for the l1_penalty is 3448968612.16 which leads to 7 nonzero weights


In [24]:
for i in range(len(final_model.get("coefficients"))):
    if final_model.coefficients['value'][i] != 0:
        print final_model.coefficients['name'][i]

(intercept)
bedrooms
bathrooms
sqft_living
sqft_living_sqrt
grade
sqft_above
